In [2]:
import pandas as pd
import numpy as np
import re

C:\Users\vladi\AppData\Local\Temp\ipykernel_14920\1346524901.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
MSAD_PERIODS = [3, 5, 10, 25, 50]

In [4]:
df = pd.read_excel('../var/stock_data.xlsx', sheet_name='Si')

In [10]:
def prepare_columns(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Renames columns (removes special symbols) and 
    '''

    df = df.rename(columns={c: re.sub("[<>]", "", c) for c in df.columns})
    df["DATE"] = pd.to_datetime(df["DATE"], format="%y%m%d")
    df["TIME"] = pd.to_datetime(df["TIME"], format="%H%M%S").dt.time
    df["DATETIME"] = pd.to_datetime(df["DATE"].astype(str) + ' ' + df["TIME"].astype(str))  
    
    
    df.drop(
        [
            "DATE",
            "TIME",
            "DATETIME_KEY",
            "TICKER",
            "PER"
        ],
        axis=1,
        inplace=True
    )

    for col in [
        "OPEN",
        "CLOSE",
        "HIGH",
        "LOW",
        "VOL"
    ]:
        df[col] = df[col].astype('float32')

    return df

def add_pivot(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Function calculates Pivot ((High + Low + Close) / 3).
    '''

    df['PIVOT'] = (df['HIGH'] + df['LOW'] + df['CLOSE']) / 3
    return df

def add_movings(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Function adds moving averages to dataframe.
    Calculates MAs by mean and standard deviation.
    '''

    for per in MSAD_PERIODS:
        df[f'MSAD_{per}_mean'] = df['PIVOT'].rolling(per).mean()
        df[f'MSAD_{per}_std'] = df['PIVOT'].rolling(per).std()
        df[f'VOL_MA_{per}_mean'] = df['VOL'].rolling(per).mean()
        df[f'VOL_MA_{per}_std'] = df['VOL'].rolling(per).std()
        
    return df

def add_fractals(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Function calculates fractals Up and Down.
    - Fractal Up, var.1:
        Candle[1] High > Candle[0] High and
        Candle[1] High > Candle[2] High.
    - Fractal Up, var.2:
        Candle[1] High == Candle[0] High and
        Candle[2] High < Candle[1] High and
        Candle[2] Close < Candle[2] Open (red candle).
    - Fractal Down, var.1:
        Candle[1] Low < Candle[0] Low and
        Candle[1] Low < Candle[2] Low.
    - Fractal Down, var.2:
        Candle[1] Low == Candle[0] Low and
        Candle[1] Low < Candle[2] Low and
        Candle[2] Close > Candle[2] Open (green candle).
    '''

    df_len = df.shape[0]
    fractals_down = [0] * df_len
    fractals_up = [0] * df_len
    
    for i in range(1, df_len-1):
        # fractal Down
        fractal_down1 = (
            (df['LOW'][i] < df['LOW'][i-1]) and
            (df['LOW'][i] < df['LOW'][i+1])
        )
        fractal_down2 = (
            (df['LOW'][i] == df['LOW'][i-1]) and
            (df['LOW'][i] < df['LOW'][i+1]) and
            (df['CLOSE'][i+1] > df['OPEN'][i+1])
        )
        if fractal_down1 or fractal_down2:
            fractals_down[i] = 1

        # fractal Up
        fractal_up1 = (
            (df['HIGH'][i] > df['HIGH'][i-1]) and
            (df['HIGH'][i] > df['HIGH'][i+1])
        )
        fractal_up2 = (
            (df['HIGH'][i] == df['HIGH'][i-1]) and
            (df['HIGH'][i] > df['HIGH'][i+1]) and
            (df['CLOSE'][i+1] < df['OPEN'][i+1])
        )
        if fractal_up1 or fractal_up2:
            fractals_up[i] = 1

    df['IS_FRACTAL_DOWN'] = fractals_down
    df['IS_FRACTAL_UP'] = fractals_up
    return df

def calc_end_correction(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Function calculates end correction.
    End correction is upper shadow for green candles and
    lower shadow for red candles.
    If Open == Close then end correction is defined by
    previous candle.
    '''

    len_df = df.shape[0]
    end_corr, end_corr_perc = [0] * len_df, [0] * len_df
    for i in range(0, len_df):
        k = 0
        while k <= i:
            corr = df['CLOSE'][i-k] - df['OPEN'][i-k]
            if corr > 0:
                end_corr[i] = df['HIGH'][i] - df['CLOSE'][i]
                end_corr_perc[i] = (
                    end_corr[i] / (df['CLOSE'][i] - df['OPEN'][i])
                ) if df['OPEN'][i] != df['CLOSE'][i] else 0
                break
            elif corr < 0:
                end_corr[i] = df['CLOSE'][i] - df['LOW'][i]
                end_corr_perc[i] = (
                    end_corr[i] / (df['OPEN'][i] - df['CLOSE'][i])
                ) if df['OPEN'][i] != df['CLOSE'][i] else 0
                break
            else:
                k += 1

    df['END_CORRECTION'] = end_corr
    df['END_CORRECTION_PERC'] = end_corr_perc
    return df

def calc_active_impulses(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Function calculates active impulses and adds data to dataframe.
    Upgoing active impulse:
        - Candle[1] Open > Candle[0] Open
        - Candle[1] Close > Candle[0] Close
        - Candle[1] Close > Candle[1] Open
        - Candle[1] Body (Close - Open) >= Candle[0] Body
    Downgoing active impulse:
        - Candle[1] Open < Candle[0] Open
        - Candle[1] Close < Candle[0] Close
        - Candle[1] Close < Candle[1] Open
        - Candle[1] Body (Open - Close) >= Candle[0] Body
    Common features for both directions:
        - Candle[1] End Correction Percent < Candle[0] End Correction Percent
        - If End Correction Percent of both candles is equal: Candle[1] End Correction < Candle[0] End Correction
        - Candle[1] End Correction <= Candle[1] Body * 0.2
    '''

    len_df = df.shape[0]
    for i in range(1, len_df):
        # upgoing active impulse
        if df['CLOSE'][i] > df['OPEN'][i]:
            is_active_impulse = (
                df['OPEN'][i] > df['OPEN'][i-1] and
                df['CLOSE'][i] > df['CLOSE'][i-1] and
                df['CLOSE'][i] - df['OPEN'][i] > df['CLOSE'][i-1] - df['OPEN'][i-1]
            )

    return df

In [11]:
d1 = calc_end_correction(df)
d1.head()

KeyError: 'CLOSE'

In [83]:
df = add_pivot(df)
df = add_movings(df)
df = add_fractals(df)
df = calc_end_correction(df)

In [84]:
df.shape

(65543, 30)

In [88]:
df.head(10)

,OPEN,HIGH,LOW,CLOSE,VOL,PIVOT,MSAD_3_mean,MSAD_3_std,VOL_MA_3_mean,VOL_MA_3_std,...,VOL_MA_25_mean,VOL_MA_25_std,MSAD_50_mean,MSAD_50_std,VOL_MA_50_mean,VOL_MA_50_std,IS_FRACTAL_DOWN,IS_FRACTAL_UP,END_CORRECTION,END_CORRECTION_PERC
DATETIME,,,,,,,,,,,,,,,,,,,,,
2023-01-03 09:00:00,70105,70500,70105,70378,17203,70327.666667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,122,0.446886
2023-01-03 09:05:00,70383,70392,70205,70286,5659,70294.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,81,0.835052
2023-01-03 09:10:00,70282,70360,70163,70296,4963,70273.000000,70298.333333,27.551971,9275.000000,6874.663046,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,64,4.571429
2023-01-03 09:15:00,70298,70350,70233,70257,3186,70280.000000,70282.444444,10.874707,4602.666667,1275.269514,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,24,0.585366
2023-01-03 09:20:00,70258,70313,70100,70122,4688,70178.333333,70243.777778,56.784518,4279.000000,956.500392,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,22,0.161765
2023-01-03 09:25:00,70120,70178,70085,70095,4355,70119.333333,70192.555556,81.272063,4076.333333,788.823385,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,10,0.400000
2023-01-03 09:30:00,70090,70174,70062,70118,5799,70118.000000,70138.555556,34.455016,4947.333333,756.124549,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,56,2.000000
2023-01-03 09:35:00,70125,70343,70121,70338,4032,70267.333333,70168.222222,85.835329,4728.666667,940.899747,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5,0.023474
2023-01-03 09:40:00,70330,70370,70192,70231,3348,70264.333333,70216.555556,85.364795,4393.000000,1264.749382,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,39,0.393939


In [47]:
df['END_CORRECTION'].value_counts()

END_CORRECTION
10      1077
15      1055
20      1040
18      1024
11      1020
        ... 
626        1
1606       1
416        1
609        1
739        1
Name: count, Length: 499, dtype: int64